## CNN Example



 
* Choose the correct _kernel_
* The kernel is the underlying environment for the Jupyter Notebook.  
* The right kernel will be built to interact with the GPUs.  It does so in a way that is (somewhat) transparent to us.
* By default, TensorFlow runs operations on all available GPU memory.
* If a TensorFlow operation has both CPU and GPU available, TensorFlow will automatically place the operation to run on a GPU device first.
* If you have more than one GPU, the GPU with the lowest ID will be selected by default. However, TensorFlow does not place operations into multiple GPUs automatically.



In [ ]:
# import sys
# print(sys.path)
import os
print(os.path)
print(os.uname().nodename)
# os.chdir("/project/hpc_training/DL_with_GPU") 

#### Load the Keras Packages

In [ ]:
import numpy as np      
from tensorflow.keras.datasets import mnist 
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout  
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
 
import matplotlib.pyplot as plt

#### Check If GPU Available

In [ ]:
import tensorflow
tensorflow.config.list_physical_devices("GPU")

#### Read in the Data

In [ ]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

print('Train: X=%s, y=%s' % (X_train.shape, Y_train.shape))
print('Test: X=%s, y=%s' % (X_test.shape, Y_test.shape))

#### Dsiplay Sample images

In [ ]:
for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(X_train[i], cmap=plt.get_cmap('gray'))
plt.show()

#### Set up the Constants

In [ ]:
numTrain = X_train.shape[0]
numTest = X_test.shape[0]
numRows = X_train.shape[1]
numCols = X_train.shape[2]
labels = set(Y_train)
input_size = numRows * numCols
numLabels = len(labels)         
np.random.seed(1234)  #for reproducibility


#### Reshape the Data


In [ ]:
X_train = X_train.reshape(numTrain, numRows, numCols, 1)
X_test = X_test.reshape(numTest, numRows, numCols, 1)

#  Scale values
X_train = X_train.astype('float32')/255    
X_test = X_test.astype('float32')/255

#  Convert labels to ‘one-hot’ vectors
Y_train = to_categorical(Y_train, numLabels)     
Y_test = to_categorical(Y_test, numLabels) 


#### Define the Model

In [ ]:
#  Set up model for tensorflow
model = Sequential()

#  First Convolution Layer
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', kernel_initializer='he_uniform', input_shape=(numRows, numCols, 1)))
model.add(MaxPooling2D(pool_size=(2,2))) 
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))                  
model.add(Flatten())         

# Fully Connected Layer
model.add(Dense(units=200, activation='relu'))

# Output Layer
model.add(Dense(numLabels, activation='softmax'))       


In [ ]:
# plot the model
plot_model(model)

#### Configure the Learning Process

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 


#### Checkpointing and Earlystopping

In [ ]:
filepath="weights.best.hdf5"    # saves the model weights to the same file if and only if the validation accuracy improves.

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')

es = EarlyStopping(monitor='val_accuracy', patience=10)   # stops training if val accuracy does not improve for 10 consecutive epochs

# logs = "./logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")    # tensorboard needs an OOD Desktop session
# tboard_callback = TensorBoard(log_dir=logs, histogram_freq=1)

callbacks_list = [checkpoint, es]    # [checkpoint, es, tboard_callback]

#### Fit the Model to the Training Data

In [ ]:
history = model.fit(X_train, Y_train, batch_size=100, epochs=30, validation_split = 0.1, callbacks=callbacks_list, verbose=1) 
score = model.evaluate(X_train, Y_train, verbose=1)   # default batch_size=32
print('\nTrain accuracy:', score[1])          


#### Save the Model

In [ ]:
model.save('~/DL_with_GPU/model.h5')

#### Plot the Learning Curve

In [ ]:
# plot learning curves
plt.title('Learning Curves')
plt.xlabel('Epoch')
plt.ylabel('Cross Entropy')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

#### Apply the Model to the Test Data

In [ ]:
loss, score = model.evaluate(X_test, Y_test, verbose=1) 
print('\nTest accuracy:', score)


#### Loading a Checkpointed NN Model

In [ ]:
# This assumes you already know the structure of the NN model since checkpointing only saves the weights.
model_2 = Sequential()
model_2.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', kernel_initializer='he_uniform', input_shape=(numRows, numCols, 1)))
model_2.add(MaxPooling2D(pool_size=(2,2))) 
model_2.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model_2.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model_2.add(MaxPooling2D((2, 2)))
model_2.add(Dropout(0.25))                  
model_2.add(Flatten())         
model_2.add(Dense(units=200, activation='relu'))
model_2.add(Dense(numLabels, activation='softmax'))

# load weights
model_2.load_weights("weights.best.hdf5")

# Compile model (required to make predictions)
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# estimate accuracy on whole dataset using loaded weights
scores = model_2.evaluate(X_test, Y_test, verbose=0)
print("%s: %.2f%%" % (model_2.metrics_names[1], scores[1]*100))

#### Loading a Saved Model

In [ ]:
from tensorflow.keras.models import load_model
model_3 = load_model('~/DL_with_GPU/model.h5')

# row_3 = X_test[-100].reshape((1,-1))
# prediction_3 = np.argmax(model.predict(row_3), axis=-1)
# print(prediction_3)
print(model_3.predict(X_test)[0])
